<div style="float:left">
    <h1 style="width:600px">Assessment</h1>
    <h3 style="width:600px">CASA0006: Data Science for Spatial Systems</h3>
    <h3 style="width:600px">Author: Andres Restrepo</h3>

</div>
<div style="float:right"><img width="100" src="https://github.com/jreades/i2p/raw/master/img/casa_logo.jpg" /></div>

MSc Urban Spatial Science<br>
Centre for Advanced Spatial Analysis CASA<br>
University College London UCL

### Size and execution time notice

**1. Pre-processing:** As the non-spatial raw data required in this research exceed the 100 megabyte limit and the download might endanger the successful execution of the notebook, the initial input of the code will be already pre-processed data. The notebook contains the code used for the cleaning and pre-processing in case future reproduction. Regarding the spatial data, there is no issue with it and it will be reading locally.

**2. Time of execution:** As the spatial clustering sections includes 6 scenarios when running the max-p method (1102 polygons), that section alone needs around 12 minutes to execute (16 GB RAM, Windows 11).



### Requered libraries

In [1]:
# Execution time measure
import timeit
start_time = timeit.default_timer()

#Preprocesing and visualizing
import os
import urllib.request
from urllib.request import urlopen
import zipfile
import geopandas as gpd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

#Clustering
from sklearn.cluster import KMeans, DBSCAN, OPTICS, AgglomerativeClustering
from sklearn import metrics
from scipy.cluster.hierarchy import dendrogram
import spopt
from spopt.region import MaxPHeuristic as MaxP
import libpysal
import warnings
import contextily
import esda

from IPython.display import Image
from IPython.core.display import HTML 

# Introduction

This research presents a proposal for analysing and clustering territorial entities in Colombia, South America at a municipality level to comprehend their heterogeneity and homogeneity from both a non-spatial and spatial perspective.


At a political and administrative level, Colombia is divided 33 state or department territorial entities (DANE, 2021). Each department consists of a group of adjacent municipalities. There is a total of 1.121municipal level entities, however, from which 1102 are truly considered municipalities, 1 insular and 18 continental non-municipalized territories (DANE, 2021).


Using non-spatial and the municipality spatial structure, there are multiple possible and cohesive department clusters that might better reflect and point out relevant shared attributes from the Colombian territory. Finding new possible municipality clusters, is a high valuable asset when formulating decentralized public policy and business initiatives that aim to address this local attributes more accurately.


# Literature review

As an initiative from the government, the Colombian National Planning Department (DNP) has conducted municipalities classification exercises related the rurality and typology. Although this research does not aim to replicate or contrast this rural and municipality typology clustering exercises, they do provide valuable references in the matter.


In the rurality case, using population density and the proportion of rural and urban population, the municipalities were classified in the following categories (Andrés et al., 2015):
1. Cities and agglomerations.
2. Intermediates.
3. Rural.
4. Dispersed rural.


Regarding the municipality topology (Sanchez, González and Villamil, 2015), using PCA  on 18 variable of 6 dimensions (Urban Regional (5), Living Conditions (1), Economic Development (4), Environmental (2), Institutional (2) and Security (4), the municipalities were grouped in 3 categories:
1. Robust development.
2. Intermediate development.
3. Incipient development.


Important to note that these previous references do not account for the spatial structure and connectivity as this research intents to.


# Research question

The aim of this research is to formulate an alternative municipality administrative division using socio-economic and geographic data.


- How could the municipalities be alternatively clustered using socio-economic and geographic data?
- Is there a more municipality cohesive department level division that better reflect the spatial structure and socio-economic attributes of the Colombian territory?


Colombia is a highly diverse country from multiple perspectives. Either from a economic and social view (Sánchez and Núñez’, 2000), agricultural productive process (Gáfaro and Pellegrina, 2022) or a (uneven) geographically outlook (Duranton, 2015) cited by (Gáfaro and Pellegrina, 2022), this significant heterogeneity might come as a challenge when formulating business initiatives and public policies at regional or local level.


By addressing the research question, this study aims to contribute to have a deeper understanding of both similarities and differences that municipalities in Colombia have, including the regional cohesion that may result from neighbouring municipalities.


# Data

The information sources for this research are the followings:

1- Terridata: It is a compilation of multiple official data sources gathered, cured, cleaned, and structured by the National Planning Department (DNP) or Colombia. The Terridata dataset includes multiple data dimensions and indicators in a multitemporal yearly and monthly time frame for every municipality, department and at the national level.

Terridata download webpage: [Terridata download web page](https://terridata.dnp.gov.co/index-app.html#/descargas)


Terridata raw data: [Raw Terridata data](https://terridata.dnp.gov.co/assets/docs/txt/dimensiones/TerriData.txt.zip)

2- Spatial data: It includes shapes of the municipality entities Colombia and the state level administrative groupings. The spatial information is generated by the National Administrative Statistics Department (DANE).

DANE Spatial data:  [DANE download web page](https://geoportal.dane.gov.co/servicios/descarga-y-metadatos/descarga-mgn-marco-geoestadistico-nacional/p)


## Reading

In the next section, the spatial and non-spatial data will be read locally. In the case of the pre-processed version of the non-spatial data, this will be done later in section: **5.2.4.4. Read pre-processed data.**

### Terridata

In [2]:
# %%time
# with zipfile.ZipFile('Data/Terri_data/TerriData.txt.zip', 'r') as zip_file:
#     with zip_file.open('TerriData.txt') as file:
#         # read the text file into a pandas dataframe
#         raw_df = pd.read_csv(file, sep = '|', thousands='.' ,decimal=',',header = None, encoding='utf-16',dtype=({0:str,2:str, 4:'category',5:'category'}),low_memory=False)

In [3]:
# # Raw data reading
# raw_df.head()

In [4]:
# # Renaming of columns
# raw_df = raw_df.rename(columns={0: 'dep_code',
#                         1: 'dep',
#                         2: 'mun_code',
#                         3: 'mun',
#                         4: 'dim',
#                         5: 'subcat',
#                         6: 'indicador',
#                         7: 'indicador_code',
#                         8: 'num_data',
#                         9: 'qua_data',
#                         10: 'year',
#                         11: 'month',
#                         12: 'source',
#                         13: 'units'})

In [5]:
raw_df.head()

NameError: name 'raw_df' is not defined

In [ ]:
# raw_df.info()

In [ ]:
# # Creationg a control copy of the raw data
# raw_df_control = raw_df.copy()

### Spatial Data

In [ ]:
# Reading spatial data from local
dep_geo = gpd.read_file('Data/Spatial/MGN2021_DPTO_POLITICO/MGN_DPTO_POLITICO.shp')
mun_geo = gpd.read_file('Data/Spatial/MGN2021_MPIO_POLITICO/MGN_MPIO_POLITICO.shp')

#### Departments spatial data

In [ ]:
#Reviewing data type
dep_geo.info()

In [ ]:
# Preview
dep_geo.head()

In [ ]:
# Geometry
dep_geo.plot() 

#### Municipalities spatial data

In [ ]:
#Reviewing data type
mun_geo.info()

In [ ]:
# Preview
mun_geo.head() 

In [ ]:
# Geometry
mun_geo.plot() 

##### Types of municipalities

In [ ]:
# Types of municipality level entities
mun_geo[['MPIO_TIPO','MPIO_CDPMP']].groupby('MPIO_TIPO').count()

In [ ]:
mun_geo[mun_geo['MPIO_TIPO']=='ISLA']

As mentioned before, from the 1121 municipality level entities, 1102 are legally considered municipalities. There are 18 non-municipalized are that will be kept in the analysis, in contrast to the San Andres Island that will be dropped later in the pre-processing sections.

## Preprocesing

### Terridata

#### Deleting national data

In [ ]:
# #Erasing the total national values
# raw_df = raw_df[raw_df.mun_code != '01001']

#### Deleting state data

In [ ]:
# # Deleting state data
# raw_df = raw_df[~raw_df['mun_code'].str.endswith('000',na=True)]

In [ ]:
# # Number of codes erased
# print('Number of municipality codes erased')
# len(raw_df_control['mun_code'].unique()) - len(raw_df['mun_code'].unique())

In [ ]:
# # Number of codes remaining
# print('Number of municipality codes remaning')
# len(raw_df['mun_code'].unique())

#### Data availability over time

As part of the exploratory data analysis, the time frame of the research and the variables of interest most be selected based on the data available. 

In [ ]:
# # Pivot table with dimentions and subcategories on the rows, years on the columns and municipalities count as aggregations method
# raw_df_dim = raw_df[(raw_df.num_data.notna())].pivot_table(index='year', values='num_data', aggfunc='count',columns='dim')

In [ ]:
# raw_df_dim.head()

Next, the columns headings are translated from Spanish to English.

In [ ]:
# # Translation of spanish dimensions to english
# english = ['Environmental',
#  '2005 Census and Dane projections',
#  'Council and citizen security',
#  'Demography and population',
#  'General description',
#  'Economy',
#  'Education',
#  'Public finances',
#  'Justice and law',
#  'Departmental performance measurement',
#  'Municipal performance measurement',
#  'Working market',
#  'Territorial planning',
#  'Poverty',
#  'General budget of the Nation',
#  'Health',
#  'Integral maritime and fluvial security',
#  'Housing and access to public services']

In [ ]:
sns.set_theme()

In [ ]:
# sns.lineplot(data=raw_df_dim, palette="tab10", linewidth=2.5)
# plt.suptitle("Terridata - Data gathered",y=1.015)
# plt.title("By Dimension")
# plt.xlabel("Year")
# plt.ylabel("Number of entries")
# plt.legend(labels=english)
# plt.legend(bbox_to_anchor=(2, 0.5),loc='center right',title='Dimension',labels=english)
# sns.set(rc={'figure.figsize':(6,4)})
# plt.savefig('EDA_Plot_Dim_Year.jpg',dpi=100,bbox_inches='tight')

In [ ]:
Image(url= "EDA_Plot_Dim_Year.jpg", width=1000, height=200)

#### Reshape data structure

In [ ]:
# # Description of the data
# raw_df.info()

In [ ]:
# #Filtering december and only numerical data
# raw_df_total = raw_df[(raw_df.month==12) & (raw_df.num_data.notna())]

In [ ]:
# raw_df_total_wide = raw_df_total.pivot(index='mun_code', columns=['dim','subcat','indicador','year'], values='num_data')

In [ ]:
# raw_df_total_wide.head()

### Spatial data

#### Departments

In [ ]:
dep_geo.info() 

In [ ]:
dep_geo.crs

In [ ]:
dep_geo = dep_geo.to_crs(21897)
dep_geo.crs

#### Municipalities

In [ ]:
mun_geo.info() 

In [ ]:
mun_geo.crs

In [ ]:
mun_geo = mun_geo.to_crs(21897)
mun_geo.crs

In [ ]:
# Calculating centroids from mun

In [ ]:
mun_geo['centroid'] = mun_geo.centroid
mun_geo['cen_lon'] = mun_geo.centroid.apply(lambda p: p.x)
mun_geo['cen_lat'] = mun_geo.centroid.apply(lambda p: p.y)

In [ ]:
mun_geo.head()

In [ ]:
# non spatial consistent with spatial data
mun_geo['MPIO_TIPO'].value_counts()

### Standardisation

#### Terridata

In [ ]:
scaler = MinMaxScaler()

In [ ]:
#raw_df_total_wide.head()

In [ ]:
#raw_df_total_wide[('Descripción general','Descripción general','Población total',2022)].max()

In [ ]:
# raw_df_total_wide_stan = pd.DataFrame(scaler.fit_transform(raw_df_total_wide.values), columns=raw_df_total_wide.columns, index=raw_df_total_wide.index)

In [ ]:
# raw_df_total_wide_stan

#### Spatial data

Only the coordinates and area.

In [ ]:
cols_to_scale = ['cen_lon','cen_lat','MPIO_NAREA']
cols_scaled = ['cen_lon_stan','cen_lat_stan','Area']
scaler.fit(mun_geo[cols_to_scale])
mun_geo[cols_scaled] = scaler.transform(mun_geo[cols_to_scale])

**Inclusion of area**

In [ ]:
mun_geo.head()

### Varliable selection

In [ ]:
# sub = [('Descripción general','Descripción general','Población total',2022),
#        ('Descripción general','Descripción general','Densidad poblacional',2022),
#        ('Demografía y población','Población desagregada por área','Población rural',2022),
#        ('Economía','Valor agregado municipal','Valor agregado per cápita',2020),
#        ('Economía','Valor agregado municipal','Valor agregado',2020)]

# # Not finding
# #('Mercado laboral','Ocupación formal','Porcentaje de personas ocupadas formalmente con respecto a la población total',2016)

# raw_df_total_wide_stan_sub = raw_df_total_wide_stan[sub]
# raw_df_total_wide_stan_sub

#### Inspecting

##### NA values

#### Descriptive statistics

In [ ]:
# #Transpose
# total_summary = raw_df_total_wide_stan_sub.describe().round(2)
# total_summary = total_summary.T
# total_summary

In [ ]:
# raw_df_total_wide_stan_sub.columns = raw_df_total_wide_stan_sub.columns.map('{0[0]}_{0[1]}_{0[2]}_{0[3]}'.format) 

In [ ]:
# raw_df_total_wide_stan_sub

In [ ]:
# raw_df_total_wide_stan_sub.info()

#### Save pre-proccesed data

In [ ]:
#raw_df_total_wide_stan_sub.to_csv('Data/Terri_data/Pre_processed/pre_processed.csv')

#### Read pre-proccesed data

In [ ]:
raw_df_total_wide_stan_sub = pd.read_csv('Data/Terri_data/Pre_processed/pre_processed.csv',dtype=({'mun_code':str}),index_col='mun_code')

In [ ]:
raw_df_total_wide_stan_sub.info()

### Merge of spatial and non spatial data

#### Non municipalized areas

In [ ]:
# Check for completeness of information in the codes

In [ ]:
len(list(raw_df_total_wide_stan_sub.index.values))  # this will always work in pandas

In [ ]:
len(list(mun_geo.MPIO_CDPMP.values))

In [ ]:
# Missing codes in Terridata information
mun_code_miss = set(list(mun_geo.MPIO_CDPMP.values)) - set(list(raw_df_total_wide_stan_sub.index.values))

In [ ]:
mun_geo[mun_geo['MPIO_CDPMP'].isin(mun_code_miss)]

**Ajustar**

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,9))

mun_geo.plot(column='MPIO_TIPO',ax=ax,legend=True)

In [ ]:
raw_df_total_wide_stan_sub.head()

In [ ]:
### Saving pre-processed data

#### Merge of datasets

In [ ]:
mun_geo_merged = pd.merge(mun_geo,raw_df_total_wide_stan_sub,how='left',left_on='MPIO_CDPMP',right_index=True)
mun_geo_merged.head()

In [ ]:
mun_geo_merged[mun_geo_merged['MPIO_CDPMP'].isin(mun_code_miss)]

**how to fill Nas in non municipilized areas?**


#### Filling variables of interest for non municipalized areas

In [ ]:
columns_to_fill = list(raw_df_total_wide_stan_sub)

In [ ]:
#Profiling to html
#profile = ProfileReport(raw_df_total_wide_stan_sub, title="Pandas Profiling Report")
#profile.to_file("Profile_raw_df_total_wide_stan_sub.html")
#profile.to_notebook_iframe()

In [ ]:
#First iteration
for col in columns_to_fill:
    for i, row in mun_geo_merged.iterrows():
        if np.isnan(row[col]):
            neighbors = mun_geo_merged[mun_geo_merged.geometry.touches(row.geometry)].copy()
            neighbors['mean'] = neighbors[col].mean()
            mean = neighbors['mean'].mean()
            mun_geo_merged.at[i, col] = mean

In [ ]:
mun_geo_merged[mun_geo_merged['MPIO_CDPMP'].isin(mun_code_miss)]

In [ ]:
mun_geo_merged[mun_geo_merged[columns_to_fill].isna().all(1)]

In [ ]:
#Second iteration
for col in columns_to_fill:
    for i, row in mun_geo_merged.iterrows():
        if np.isnan(row[col]):
            neighbors = mun_geo_merged[mun_geo_merged.geometry.touches(row.geometry)].copy()
            neighbors['mean'] = neighbors[col].mean()
            mean = neighbors['mean'].mean()
            mun_geo_merged.at[i, col] = mean

#San Andres and Providencia Island

In [ ]:
mun_geo_merged[mun_geo_merged[columns_to_fill].isna().all(1)]

In [ ]:
mun_geo_merged[mun_geo_merged.MPIO_CDPMP=='88564']

In [ ]:
mun_geo_merged[mun_geo_merged.MPIO_CDPMP=='88001']

In [ ]:
# Removing San Andres and Providencia Island
mun_geo_merged.drop(mun_geo_merged[mun_geo_merged.MPIO_CDPMP=='88001'].index,inplace=True)
mun_geo_merged.drop(mun_geo_merged[mun_geo_merged.MPIO_CDPMP=='88564'].index,inplace=True)

In [ ]:
mun_geo_merged_clean_name = mun_geo_merged.rename(columns={'Descripción general_Descripción general_Población total_2022': 'Population - 2022',
                        'Descripción general_Descripción general_Densidad poblacional_2022': 'Population density - 2022',
                        'Demografía y población_Población desagregada por área_Población rural_2022': 'Rural population - 2022',
                        'Economía_Valor agregado municipal_Valor agregado per cápita_2020': 'Per capita added value - 2020',
                        'Economía_Valor agregado municipal_Valor agregado_2020': 'Added value - 2020',
                        'cen_lon_stan': 'Centroid longitude',
                        'cen_lat_stan': 'Centroid latitude'})

In [ ]:
mun_geo_merged_clean_name.head()

In [ ]:
mun_geo_merged_clean_name.info()

In [ ]:
mun_geo_merged_clean_name_control = mun_geo_merged_clean_name.copy()

In [ ]:
mun_geo_merged_clean_name_control.shape

In [ ]:
mun_geo_merged_clean_name_control.head()

In [ ]:
#Including standarized centroids, non spatial and Area as variabels of interest
list(mun_geo_merged_clean_name_control.columns)[-8:]

In [ ]:
cols_to_plot = list(mun_geo_merged_clean_name_control.columns)[-8:]

In [ ]:
#mun_geo_merged_clean_name.head()

In [ ]:
mun_geo_merged_clean_name_ana = mun_geo_merged_clean_name[cols_to_plot]

In [ ]:
mun_geo_merged_clean_name_ana_hist = mun_geo_merged_clean_name_ana.hist(figsize = (20,10),bins=75)
plt.suptitle('Histograms of variables of interest')
#ax.set_title("test")
plt.subplots_adjust(hspace=0.4, wspace=0.4)
plt.show()

# Methodology

## Clustering

### Function for ploting clusters centroids

In [ ]:
# Radar plot of the cluster centroids
# adapted from this tutorial: https://towardsdatascience.com/how-to-make-stunning-radar-charts-with-python-implemented-in-matplotlib-and-plotly-91e21801d8ca
def radar_plot_cluster_centroids(method,df_cluster_centroid):
    # parameters
    # df_cluster_centroid: a dataframe with rows representing a cluster centroid and columns representing variables
    
    # add an additional element to both categories and restaurants that’s identical to the first item
    # manually 'close' the line
    categories = df_cluster_centroid.columns.values.tolist()
    categories = [*categories, categories[0]]
    
    label_loc = np.linspace(start=0, stop=2 * np.pi, num=len(categories))
    
    plt.figure(figsize=(12, 8))
    plt.subplot(polar=True)
    for index, row in df_cluster_centroid.iterrows():
        centroid = row.tolist()
        centroid = [*centroid, centroid[0]]
        label = "Cluster {}".format(index)
        plt.plot(label_loc, centroid, label=label)
    plt.title('Cluster centroid comparison'+' - '+ method, size=20, y=1.05)
    lines, labels = plt.thetagrids(np.degrees(label_loc), labels=categories)
    plt.legend(bbox_to_anchor=(1.4, 0.5),loc='center right')
    plt.show()

### Non spatial clustering

#### DBSCAN

**Should take only rates in the variable?**

In [ ]:
list(mun_geo_merged_clean_name_control.columns)[-6:]

In [ ]:
mun_geo_merged_clean_name_control

In [ ]:
non_spatial = list(mun_geo_merged_clean_name_control.columns)[-6:]

In [ ]:
minPts = 7 # we set minPts as normed.shape[1] + 1 
epsilon = 0.05
dbsc_non = DBSCAN(eps=epsilon, min_samples=minPts)
dbsc_non.fit(mun_geo_merged_clean_name[non_spatial])

# We now have our DBSCAN object created, and we can extract the groups it has identified. We do this using the `.labels_` method.

mun_geo_merged_clean_name['non_spatial_dbscan'] = dbsc_non.labels_

In [ ]:
# Summary
pd.Series(dbsc_non.labels_).value_counts()

In [ ]:
len(mun_geo_merged_clean_name['non_spatial_dbscan'].unique())

In [ ]:
mun_geo_merged_clean_name[['non_spatial_dbscan','MPIO_CDPMP']].groupby('non_spatial_dbscan').count()

In [ ]:
mun_geo_merged_clean_name.groupby('non_spatial_dbscan')['MPIO_CDPMP'].count().mean()

In [ ]:
metrics.silhouette_score(mun_geo_merged_clean_name_control[non_spatial], dbsc_non.labels_)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,9))
mun_geo_merged_clean_name.plot(column='non_spatial_dbscan',ax=ax,legend=True,linewidth=0.5,categorical=True)

**hierawuical plot**

**Twin partnership cities**

In [ ]:
#mun_geo_merged_clean_name_control[non_spatial]

Clustering comparasion

In [ ]:
df_dbscan_non = mun_geo_merged_clean_name_control[non_spatial].copy()
df_dbscan_non['non_spatial_dbscan'] = dbsc_non.labels_
df_dbscan_non_centroid = df_dbscan_non.groupby('non_spatial_dbscan').mean()
# drop the outlier
#df_dbscan_non_centroid.drop(-1, inplace=True)
# df_dbscan_centroid.reset_index()
radar_plot_cluster_centroids('Non spatial DBSCAN',df_dbscan_non_centroid)
#plt.title('Non spatial cluster centroid comparison', size=20, y=1.05)

In [ ]:
df_dbscan_non_centroid

In [ ]:
#Silhouette Score
metrics.silhouette_score(mun_geo_merged_clean_name_control[non_spatial], dbsc_non.labels_)

In [ ]:
mun_geo_merged_clean_name[mun_geo_merged_clean_name.non_spatial_dbscan==1]

In [ ]:
#mun_geo_merged_clean_name[mun_geo_merged_clean_name.non_spatial_cluster==-1]

**Urban bised variables**

#### Kmeans

**Prevent randomness in Kmeans**

In [ ]:
k_cluster = 4
random_seed = 1
kmeans_method_non_spatial = KMeans(n_clusters=k_cluster,random_state=random_seed, n_init=50)
kmeans_method_non_spatial.fit(mun_geo_merged_clean_name[non_spatial])

In [ ]:
mun_geo_merged_clean_name['non_spatial_kmeans'] = kmeans_method_non_spatial.labels_

Clustering comparasion

In [ ]:
df_kmeans_non_spatial = mun_geo_merged_clean_name_control[non_spatial].copy()
df_kmeans_non_spatial['non_spatial_kmeans'] = kmeans_method_non_spatial.labels_
df_kmeans_non_spatial_centroid = df_kmeans_non_spatial.groupby('non_spatial_kmeans').mean()
# drop the outlier
#df_kmeans_non_spatial_centroid.drop(-1, inplace=True)
# df_dbscan_centroid.reset_index()
radar_plot_cluster_centroids('Non spatial Kmeans',df_kmeans_non_spatial_centroid)

In [ ]:
df_kmeans_non_spatial_centroid

In [ ]:
metrics.silhouette_score(mun_geo_merged_clean_name_control[non_spatial], kmeans_method_non_spatial.labels_)

Right number of clusters

In [ ]:
# calculate SSE for a range of number of cluster
list_SSE_non = []
min_k = 1
max_k = 10
range_k = range(min_k, max_k+1)
for i in range_k:
    km_non = KMeans(
        n_clusters=i, init='random',
        n_init=50, max_iter=300,
        tol=1e-04, random_state=1
    )
    km_non.fit(mun_geo_merged_clean_name[non_spatial])
    # inertia is a concept from physics. Roughly it means SSE of clustering.
    list_SSE_non.append(km_non.inertia_)

# plot
plt.plot(range_k, list_SSE_non, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()

Interpratition: 3 clusters?

##### Reviewed

In [ ]:
k_cluster = 3
random_seed = 1
kmeans_method_non_spatial_re = KMeans(n_clusters=k_cluster,random_state=random_seed, n_init=50)
kmeans_method_non_spatial_re.fit(mun_geo_merged_clean_name[non_spatial])

In [ ]:
mun_geo_merged_clean_name['non_spatial_kmeans_re'] = kmeans_method_non_spatial_re.labels_

Clustering comparasion

In [ ]:
df_kmeans_non_spatial_re = mun_geo_merged_clean_name_control[non_spatial].copy()
df_kmeans_non_spatial_re['non_spatial_kmeans_re'] = kmeans_method_non_spatial_re.labels_
df_kmeans_non_spatial_re_centroid = df_kmeans_non_spatial_re.groupby('non_spatial_kmeans_re').mean()
# drop the outlier
#df_kmeans_non_spatial_centroid.drop(-1, inplace=True)
# df_dbscan_centroid.reset_index()
radar_plot_cluster_centroids('Non spatial Kmeans rerun',df_kmeans_non_spatial_re_centroid)

In [ ]:
df_kmeans_non_spatial_re_centroid

In [ ]:
metrics.silhouette_score(mun_geo_merged_clean_name_control[non_spatial], kmeans_method_non_spatial_re.labels_)

Slight improvement

#### Hierarchical clustering

The code of **plot_dendrogram** is based on the practical material, which is largely based on this [tutorial](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py).

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, leaf_rotation=90., **kwargs)

In [ ]:
agg_cluster_non = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(mun_geo_merged_clean_name_control[non_spatial])

In [ ]:
ax = plt.gca()
plt.title("Hierarchical Clustering Dendrogram - Non spatial")
# plot the top three levels of the dendrogram
plot_dendrogram(agg_cluster_non, truncate_mode="level", p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.ylabel('distance')
plt.hlines(2.65, ax.get_xlim()[0], ax.get_xlim()[1], linestyle='dashed', color='k')
plt.hlines(2, ax.get_xlim()[0], ax.get_xlim()[1], linestyle='dashed', color='k')
plt.hlines(1.75, ax.get_xlim()[0], ax.get_xlim()[1], linestyle='dashed', color='c')
plt.show()

The distance between 3 and 4 clusters, is greater than the distacne between the 4 and 5 clusters. This indicates that it is preferable to keep the 4 clusters, instead of 5.

**Attention**

In [ ]:
# Storing clusters lablel
mun_geo_merged_clean_name['non_spatial_hierarchical'] = agg_cluster_non.labels_

##### Reviewed

In [ ]:
agg_cluster_non_re = AgglomerativeClustering(distance_threshold=None, n_clusters=4).fit(mun_geo_merged_clean_name_control[non_spatial])

In [ ]:
# Storing re cluster labels
mun_geo_merged_clean_name['non_spatial_hierarchical_4'] = agg_cluster_non_re.labels_

In [ ]:
#Silhouette Score
metrics.silhouette_score(mun_geo_merged_clean_name_control[non_spatial], agg_cluster_non_re.labels_)

In [ ]:
mun_geo_merged_clean_name.head()

### Partially spatial clustering

In [ ]:
list(mun_geo_merged_clean_name_control.columns)[-8:]

In [ ]:
part_spatial = list(mun_geo_merged_clean_name_control.columns)[-8:]

In [ ]:
metrics.silhouette_score(mun_geo_merged_clean_name_control[part_spatial], mun_geo_merged_clean_name_control['DPTO_CCDGO'])

#### DBSCAN

**DBSCAN on non spatial and then take the labels to geo pandas**

**Or subset the columns for the non spatial clustering**

In [ ]:
minPts = 9 # we set minPts as normed.shape[1] + 1 
epsilon = 0.05
dbsc_part = DBSCAN(eps=epsilon, min_samples=minPts)
dbsc_part.fit(mun_geo_merged_clean_name[part_spatial])

# We now have our DBSCAN object created, and we can extract the groups it has identified. We do this using the `.labels_` method.

mun_geo_merged_clean_name['part_spatial_dbscan'] = dbsc_part.labels_

In [ ]:
# Summary
pd.Series(dbsc_part.labels_).value_counts()

**Move maps to the end of the process**

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,9))
mun_geo_merged_clean_name.plot(column='part_spatial_dbscan',categorical=True,ax=ax,legend=True,linewidth=0.5)

Clustering comparasion

In [ ]:
#part_spatial

In [ ]:
df_dbscan_part = mun_geo_merged_clean_name_control[part_spatial].copy()
df_dbscan_part['part_spatial_dbscan'] = dbsc_part.labels_
df_dbscan_centroid_part = df_dbscan_part.groupby('part_spatial_dbscan').mean()
# drop the outlier
#df_dbscan_centroid1.drop(-1, inplace=True)
# df_dbscan_centroid.reset_index()
radar_plot_cluster_centroids('Partially spatial DBSCAN',df_dbscan_centroid_part)

In [ ]:
df_dbscan_centroid_part

In [ ]:
#Silhouette Score
metrics.silhouette_score(mun_geo_merged_clean_name_control[part_spatial], dbsc_part.labels_)

Overlaping of clusters

In [ ]:
#mun_geo_merged_clean_name[mun_geo_merged_clean_name.part_spatial_cluster==-1]

#### Kmeans

**Prevent randomness in Kmeans**

In [ ]:
k_cluster = 4
random_seed = 1
kmeans_method_part_spatial = KMeans(n_clusters=k_cluster,random_state=random_seed, n_init=50)
kmeans_method_part_spatial.fit(mun_geo_merged_clean_name[part_spatial])

In [ ]:
mun_geo_merged_clean_name['part_spatial_kmeans'] = kmeans_method_part_spatial.labels_

Clustering comparasion

In [ ]:
df_kmeans_part_spatial = mun_geo_merged_clean_name_control[part_spatial].copy()
df_kmeans_part_spatial['part_spatial_kmeans'] = kmeans_method_part_spatial.labels_
df_kmeans_part_spatial_centroid = df_kmeans_part_spatial.groupby('part_spatial_kmeans').mean()
# drop the outlier
#df_kmeans_non_spatial_centroid.drop(-1, inplace=True)
# df_dbscan_centroid.reset_index()
radar_plot_cluster_centroids('Partially spatial Kmeans',df_kmeans_part_spatial_centroid)

In [ ]:
df_kmeans_part_spatial_centroid

In [ ]:
metrics.silhouette_score(mun_geo_merged_clean_name_control[part_spatial], kmeans_method_part_spatial.labels_)

Improvement compared to non spatial DBSCAN.

Right number of clusters

In [ ]:
# calculate SSE for a range of number of cluster
list_SSE_part = []
min_k = 1
max_k = 10
range_k = range(min_k, max_k+1)
for i in range_k:
    km_part = KMeans(
        n_clusters=i, init='random',
        n_init=50, max_iter=300,
        tol=1e-04, random_state=1
    )
    km_part.fit(mun_geo_merged_clean_name[part_spatial])
    # inertia is a concept from physics. Roughly it means SSE of clustering.
    list_SSE_part.append(km_part.inertia_)

# plot
plt.plot(range_k, list_SSE_part, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()

Interpratition: 3 clusters?

##### Reviewed

In [ ]:
k_cluster = 3
random_seed = 1
kmeans_method_part_spatial_re = KMeans(n_clusters=k_cluster,random_state=random_seed, n_init=50)
kmeans_method_part_spatial_re.fit(mun_geo_merged_clean_name[part_spatial])

In [ ]:
mun_geo_merged_clean_name['part_spatial_kmeans_re'] = kmeans_method_part_spatial_re.labels_

Clustering comparasion

In [ ]:
df_kmeans_part_spatial_re = mun_geo_merged_clean_name_control[part_spatial].copy()
df_kmeans_part_spatial_re['part_spatial_kmeans_re'] = kmeans_method_part_spatial_re.labels_
df_kmeans_part_spatial_re_centroid = df_kmeans_part_spatial_re.groupby('part_spatial_kmeans_re').mean()
# drop the outlier
#df_kmeans_non_spatial_centroid.drop(-1, inplace=True)
# df_dbscan_centroid.reset_index()
radar_plot_cluster_centroids('Partially spatial Kmeans rerun',df_kmeans_part_spatial_re_centroid)

In [ ]:
df_kmeans_part_spatial_re_centroid

In [ ]:
metrics.silhouette_score(mun_geo_merged_clean_name_control[part_spatial], kmeans_method_part_spatial_re.labels_)

Slight improvement compared same method.

#### Hierarchical clustering

In [ ]:
agg_cluster_part = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(mun_geo_merged_clean_name_control[part_spatial])

In [ ]:
ax = plt.gca()
plt.title("Hierarchical Clustering Dendrogram - Partially spatial")
# plot the top three levels of the dendrogram
plot_dendrogram(agg_cluster_part, truncate_mode="level", p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.ylabel('distance')
plt.hlines(4.4, ax.get_xlim()[0], ax.get_xlim()[1], linestyle='dashed', color='k')
plt.hlines(3.5, ax.get_xlim()[0], ax.get_xlim()[1], linestyle='dashed', color='k')
plt.hlines(2.9, ax.get_xlim()[0], ax.get_xlim()[1], linestyle='dashed', color='c')
plt.show()

The distance between 3 and 4 clusters, is greater than the distacne between the 4 and 5 clusters. This indicates that it is preferable to keep the 4 clusters, instead of 5.

In [ ]:
# Storing clusters lablel
mun_geo_merged_clean_name['part_spatial_hierarchical'] = agg_cluster_part.labels_

##### Reviewed

In [ ]:
agg_cluster_part_re = AgglomerativeClustering(distance_threshold=None, n_clusters=4).fit(mun_geo_merged_clean_name_control[part_spatial])

In [ ]:
# Storing re cluster labels
mun_geo_merged_clean_name['part_spatial_hierarchical_4'] = agg_cluster_part_re.labels_

In [ ]:
# Silhouette score
metrics.silhouette_score(mun_geo_merged_clean_name_control[part_spatial], agg_cluster_part_re.labels_)

In [ ]:
type(mun_geo_merged_clean_name)

### Spatial clustering - Max-p method

**Model variation in mun count and population thershold**

In [ ]:
#warnings.filterwarnings("ignore")

In [ ]:
# Non spatial variables
non_spatial

In [ ]:
# Non spatial variables to be considered in the spatial clustering
attrs_name = non_spatial

In [ ]:
%%time
# Caluclate the spatial weight object
w = libpysal.weights.Queen.from_dataframe(mun_geo_merged_clean_name)

#### Population threshold

##### Case 1

In [ ]:
# Maximum popupation unscaled value
unscaled_max_pop= 7901653.0

In [ ]:
# Threshols based on a minimum population value of 100000, in terms of maximum population value
threshold_100k_pop = 100000/unscaled_max_pop
print(threshold_100k_pop)

In [ ]:
# Scaled 2022 Population variable
threshold_name_pop = 'Population - 2022'

In [ ]:
%%time
RANDOM_SEED = 123456
np.random.seed(RANDOM_SEED)
model_max_p_pop_1 = MaxP(mun_geo_merged_clean_name, w, attrs_name, threshold_name_pop, threshold_100k_pop, top_n=3)
model_max_p_pop_1.solve()

In [ ]:
mun_geo_merged_clean_name['spatial_max_p_pop_1'] = model_max_p_pop_1.labels_

In [ ]:
#Number of regions
model_max_p_pop_1.p

In [ ]:
mun_geo_merged_clean_name[['spatial_max_p_pop_1','MPIO_CDPMP']].groupby('spatial_max_p_pop_1').count().sort_values(by='MPIO_CDPMP',ascending=False)

In [ ]:
# Histogram of Clustered municipalities
mun_geo_merged_clean_name[['spatial_max_p_pop_1','MPIO_CDPMP']].groupby('spatial_max_p_pop_1').count().hist()

In [ ]:
# Descriptive statistics of Clustered municipalities
mun_geo_merged_clean_name[['spatial_max_p_pop_1','MPIO_CDPMP']].groupby('spatial_max_p_pop_1').count().describe().T

In [ ]:
mun_geo_merged_clean_name.plot(column='spatial_max_p_pop_1', categorical=True, figsize=(12,13),linewidth=0.5)

In [ ]:
# Adjust
# mun_geo_merged_clean_name[mun_geo_merged_clean_name.spatial_max_p_single==]

In [ ]:
#set(model_max_p_single.labels_)

##### Case 2

In [ ]:
threshold_500k_pop = 500000/unscaled_max_pop
print(threshold_500k_pop)

In [ ]:
%%time
RANDOM_SEED = 123456
np.random.seed(RANDOM_SEED)
model_max_p_pop_2 = MaxP(mun_geo_merged_clean_name, w, attrs_name, threshold_name_pop, threshold_500k_pop, top_n=3)
model_max_p_pop_2.solve()

In [ ]:
mun_geo_merged_clean_name['spatial_max_p_pop_2'] = model_max_p_pop_2.labels_

In [ ]:
#Number of regions
model_max_p_pop_2.p

In [ ]:
mun_geo_merged_clean_name[['spatial_max_p_pop_2','MPIO_CDPMP']].groupby('spatial_max_p_pop_2').count().sort_values(by='MPIO_CDPMP',ascending=False)

In [ ]:
# Histogram of Clustered municipalities
mun_geo_merged_clean_name[['spatial_max_p_pop_2','MPIO_CDPMP']].groupby('spatial_max_p_pop_2').count().hist()

In [ ]:
# Descriptive statistics of Clustered municipalities
mun_geo_merged_clean_name[['spatial_max_p_pop_2','MPIO_CDPMP']].groupby('spatial_max_p_pop_2').count().describe().T

In [ ]:
mun_geo_merged_clean_name.plot(column='spatial_max_p_pop_2', categorical=True, figsize=(12,13),linewidth=0.5)

In [ ]:
# Adjust
# mun_geo_merged_clean_name[mun_geo_merged_clean_name.spatial_max_p_single==]

In [ ]:
#set(model_max_p_single.labels_)

##### Case 3

In [ ]:
threshold_1000k_pop = 1000000/unscaled_max_pop
print(threshold_1000k_pop)

In [ ]:
%%time
RANDOM_SEED = 123456
np.random.seed(RANDOM_SEED)
model_max_p_pop_3 = MaxP(mun_geo_merged_clean_name, w, attrs_name, threshold_name_pop, threshold_1000k_pop, top_n=3)
model_max_p_pop_3.solve()

In [ ]:
mun_geo_merged_clean_name['spatial_max_p_pop_3'] = model_max_p_pop_3.labels_

In [ ]:
#Number of regions
model_max_p_pop_3.p

In [ ]:
mun_geo_merged_clean_name[['spatial_max_p_pop_3','MPIO_CDPMP']].groupby('spatial_max_p_pop_3').count().sort_values(by='MPIO_CDPMP',ascending=False)

In [ ]:
# Histogram of Clustered municipalities
mun_geo_merged_clean_name[['spatial_max_p_pop_3','MPIO_CDPMP']].groupby('spatial_max_p_pop_3').count().hist()

In [ ]:
# Descriptive statistics of Clustered municipalities
mun_geo_merged_clean_name[['spatial_max_p_pop_3','MPIO_CDPMP']].groupby('spatial_max_p_pop_3').count().describe().T

In [ ]:
mun_geo_merged_clean_name.plot(column='spatial_max_p_pop_3', categorical=True, figsize=(12,13),linewidth=0.5)

#### Municipality threshold

##### Case 1

In [ ]:
threshold_5_mun = 5

In [ ]:
mun_geo_merged_clean_name['count'] = 1

In [ ]:
threshold_name_mun = 'count'

In [ ]:
%%time
RANDOM_SEED = 123456
np.random.seed(RANDOM_SEED)
model_max_p_mun_1 = MaxP(mun_geo_merged_clean_name, w, attrs_name, threshold_name_mun, threshold_5_mun, top_n=3)
model_max_p_mun_1.solve()

In [ ]:
mun_geo_merged_clean_name['spatial_max_p_mun_1'] = model_max_p_mun_1.labels_

In [ ]:
#Number of regions
model_max_p_mun_1.p

In [ ]:
mun_geo_merged_clean_name[['spatial_max_p_mun_1','MPIO_CDPMP']].groupby('spatial_max_p_mun_1').count().sort_values(by='MPIO_CDPMP',ascending=False)

In [ ]:
# Histogram of Clustered municipalities
mun_geo_merged_clean_name[['spatial_max_p_mun_1','MPIO_CDPMP']].groupby('spatial_max_p_mun_1').count().hist()

In [ ]:
# Descriptive statistics of Clustered municipalities
mun_geo_merged_clean_name[['spatial_max_p_mun_1','MPIO_CDPMP']].groupby('spatial_max_p_mun_1').count().describe().T

In [ ]:
mun_geo_merged_clean_name.plot(column='spatial_max_p_mun_1', categorical=True, figsize=(12,13), cmap='Paired',linewidth=0.5)

##### Case 2

In [ ]:
threshold_15_mun = 15

In [ ]:
%%time
RANDOM_SEED = 123456
np.random.seed(RANDOM_SEED)
model_max_p_mun_2 = MaxP(mun_geo_merged_clean_name, w, attrs_name, threshold_name_mun, threshold_15_mun, top_n=3)
model_max_p_mun_2.solve()

In [ ]:
mun_geo_merged_clean_name['spatial_max_p_mun_2'] = model_max_p_mun_2.labels_

In [ ]:
#Number of regions
model_max_p_mun_2.p

In [ ]:
mun_geo_merged_clean_name[['spatial_max_p_mun_2','MPIO_CDPMP']].groupby('spatial_max_p_mun_2').count().sort_values(by='MPIO_CDPMP',ascending=False)

In [ ]:
# Histogram of Clustered municipalities
mun_geo_merged_clean_name[['spatial_max_p_mun_2','MPIO_CDPMP']].groupby('spatial_max_p_mun_2').count().hist()

In [ ]:
# Descriptive statistics of Clustered municipalities
mun_geo_merged_clean_name[['spatial_max_p_mun_2','MPIO_CDPMP']].groupby('spatial_max_p_mun_2').count().describe().T

In [ ]:
mun_geo_merged_clean_name.plot(column='spatial_max_p_mun_2', categorical=True, figsize=(12,13), cmap='Paired',linewidth=0.5)

##### Case 3

In [ ]:
threshold_25_mun = 25

In [ ]:
%%time
RANDOM_SEED = 123456
np.random.seed(RANDOM_SEED)
model_max_p_mun_3 = MaxP(mun_geo_merged_clean_name, w, attrs_name, threshold_name_mun, threshold_25_mun, top_n=3)
model_max_p_mun_3.solve()

In [ ]:
mun_geo_merged_clean_name['spatial_max_p_mun_3'] = model_max_p_mun_3.labels_

In [ ]:
#Number of regions
model_max_p_mun_3.p

In [ ]:
mun_geo_merged_clean_name[['spatial_max_p_mun_3','MPIO_CDPMP']].groupby('spatial_max_p_mun_3').count().sort_values(by='MPIO_CDPMP',ascending=False)

In [ ]:
# Histogram of Clustered municipalities
mun_geo_merged_clean_name[['spatial_max_p_mun_3','MPIO_CDPMP']].groupby('spatial_max_p_mun_3').count().hist()

In [ ]:
# Descriptive statistics of Clustered municipalities
mun_geo_merged_clean_name[['spatial_max_p_mun_3','MPIO_CDPMP']].groupby('spatial_max_p_mun_3').count().describe().T

In [ ]:
mun_geo_merged_clean_name.plot(column='spatial_max_p_mun_3', categorical=True, figsize=(12,13),linewidth=0.5)

## Results

Include matriz with silhoutte scores /
Geographic outputs with every method

### Partially and non spatial clustering

In [ ]:
metrics.silhouette_score(mun_geo_merged_clean_name_control[non_spatial], mun_geo_merged_clean_name_control['DPTO_CCDGO'])

In [ ]:
metrics.silhouette_score(mun_geo_merged_clean_name_control[part_spatial], mun_geo_merged_clean_name_control['DPTO_CCDGO'])

In [ ]:
# Create dictionaty with the results variables of interest
results = {'Method': ['DBSCAN', 'DBSCAN', 'Kmeans', 'Kmeans','Hierarchical clustering', 'Hierarchical clustering'],
          'Type': ['Non spatial', 'Partially spatial','Non spatial', 'Partially spatial','Non spatial', 'Partially spatial']}


df_results = pd.DataFrame(results)

method = ['non_spatial_dbscan','part_spatial_dbscan','non_spatial_kmeans_re','part_spatial_kmeans_re','non_spatial_hierarchical_4','part_spatial_hierarchical_4']

num_clusters = []
mun_mean = []
mun_min = []
mun_max = []


for m in method:
    n_clusters = len(mun_geo_merged_clean_name[m].unique())
    m_mean = mun_geo_merged_clean_name.groupby(m)['MPIO_CDPMP'].count().mean()
    m_min = mun_geo_merged_clean_name.groupby(m)['MPIO_CDPMP'].count().min()
    m_max = mun_geo_merged_clean_name.groupby(m)['MPIO_CDPMP'].count().max()
    
    num_clusters.append(n_clusters)
    mun_mean.append(m_mean)
    mun_min.append(m_min)
    mun_max.append(m_max)

# add the results to a new column of the DataFrame
df_results['Number of clusters'] = num_clusters
df_results['Municipalities mean'] = mun_mean
df_results['Municipalities minimum'] = mun_min
df_results['Municipalities maximum'] = mun_max

df_results['Silhouette score'] = [metrics.silhouette_score(mun_geo_merged_clean_name_control[non_spatial], dbsc_non.labels_),
                                 metrics.silhouette_score(mun_geo_merged_clean_name_control[part_spatial], dbsc_part.labels_),
                                 metrics.silhouette_score(mun_geo_merged_clean_name_control[non_spatial], kmeans_method_non_spatial_re.labels_),
                                 metrics.silhouette_score(mun_geo_merged_clean_name_control[part_spatial], kmeans_method_part_spatial_re.labels_),
                                 metrics.silhouette_score(mun_geo_merged_clean_name_control[non_spatial], agg_cluster_non_re.labels_),
                                 metrics.silhouette_score(mun_geo_merged_clean_name_control[part_spatial], agg_cluster_part_re.labels_)]
df_results

#### DBSCAN Clustering

In [ ]:
fig, [ax1,ax2,ax3] = plt.subplots(1, 3, sharey=False, tight_layout=True,figsize=(16, 6),linewidth=0.5)
plt.suptitle("DBSCAN clustering",fontsize=22)

# Clustering results
mun_geo_merged_clean_name.plot(column='DPTO_CCDGO', categorical=True,linewidth=0.5,ax=ax1)
mun_geo_merged_clean_name.plot(column='non_spatial_dbscan', categorical=True,linewidth=0.5,ax=ax2)
mun_geo_merged_clean_name.plot(column='part_spatial_dbscan', categorical=True,linewidth=0.5,ax=ax3)

# Titles
ax1.set_title("Current clusters",fontsize=18)
ax2.set_title("Non spatial",fontsize=18)
ax3.set_title("Part spatial",fontsize=18)

# Setting off axis ticks
ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax3.set_xticklabels([])
ax1.set_yticklabels([])
ax2.set_yticklabels([])
ax3.set_yticklabels([])

# X axis labels
ax1.set_xlabel("K = "+ str(len(mun_geo_merged_clean_name['DPTO_CCDGO'].unique())),fontsize=18)
ax2.set_xlabel("K = "+ str(df_results['Number of clusters'][0]),fontsize=18)
ax3.set_xlabel("K = "+ str(df_results['Number of clusters'][1]),fontsize=18)

plt.savefig('Results_DBSCAN.jpg',dpi=300,bbox_inches='tight')

**interpratation**

#### Kmeans Clustering

In [ ]:
fig, [ax1,ax2,ax3] = plt.subplots(1, 3, sharey=False, tight_layout=True,figsize=(16, 6),linewidth=0.5)
plt.suptitle("Kmeans clustering",fontsize=22)

# Clustering results
mun_geo_merged_clean_name.plot(column='DPTO_CCDGO', categorical=True,linewidth=0.5,ax=ax1)
mun_geo_merged_clean_name.plot(column='non_spatial_kmeans_re', categorical=True,linewidth=0.5,ax=ax2)
mun_geo_merged_clean_name.plot(column='part_spatial_kmeans_re', categorical=True,linewidth=0.5,ax=ax3)

# Titles
ax1.set_title("Current clusters",fontsize=18)
ax2.set_title("Non spatial",fontsize=18)
ax3.set_title("Part spatial",fontsize=18)

# Setting off axis ticks
ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax3.set_xticklabels([])
ax1.set_yticklabels([])
ax2.set_yticklabels([])
ax3.set_yticklabels([])

# X axis labels
ax1.set_xlabel("K = "+ str(len(mun_geo_merged_clean_name['DPTO_CCDGO'].unique())),fontsize=18)
ax2.set_xlabel("K = "+ str(df_results['Number of clusters'][2]),fontsize=18)
ax3.set_xlabel("K = "+ str(df_results['Number of clusters'][3]),fontsize=18)

plt.savefig('Results_Kmeans.jpg',dpi=300,bbox_inches='tight')

**interpratation**

#### Hierarchical clustering

In [ ]:
fig, [ax1,ax2,ax3] = plt.subplots(1, 3, sharey=False, tight_layout=True,figsize=(16, 6),linewidth=0.5)
plt.suptitle("Hierarchical clustering",fontsize=22)

# Clustering results
mun_geo_merged_clean_name.plot(column='DPTO_CCDGO', categorical=True,linewidth=0.5,ax=ax1)
mun_geo_merged_clean_name.plot(column='non_spatial_hierarchical_4', categorical=True,linewidth=0.5,ax=ax2)
mun_geo_merged_clean_name.plot(column='part_spatial_hierarchical_4', categorical=True,linewidth=0.5,ax=ax3)

# Titles
ax1.set_title("Current clusters",fontsize=18)
ax2.set_title("Non spatial",fontsize=18)
ax3.set_title("Part spatial",fontsize=18)

# Setting off axis ticks
ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax3.set_xticklabels([])
ax1.set_yticklabels([])
ax2.set_yticklabels([])
ax3.set_yticklabels([])

# X axis labels
ax1.set_xlabel("K = "+ str(len(mun_geo_merged_clean_name['DPTO_CCDGO'].unique())),fontsize=18)
ax2.set_xlabel("K = "+ str(df_results['Number of clusters'][4]),fontsize=18)
ax3.set_xlabel("K = "+ str(df_results['Number of clusters'][5]),fontsize=18)

plt.savefig('Results_Hierarchical.jpg',dpi=300,bbox_inches='tight')

**interpratation**

### Spatial clustering

In [ ]:
esda.path_silhouette(mun_geo_merged_clean_name[non_spatial].values,mun_geo_merged_clean_name['DPTO_CCDGO'], w).mean()

In [ ]:
# Create dictionaty with the results variables of interest
spatial_results = {'Variaple threshold': [threshold_name_pop, threshold_name_pop, threshold_name_pop, 'Number of municipalities','Number of municipalities', 'Number of municipalities'],
          'Threshold value': [100000, 500000,1000000, threshold_5_mun,threshold_15_mun, threshold_25_mun]}

df_spatial_results = pd.DataFrame(spatial_results)
df_silhouette = pd.DataFrame()
# Adjust
spatial_method = ['spatial_max_p_pop_1','spatial_max_p_pop_2','spatial_max_p_pop_3','spatial_max_p_mun_1','spatial_max_p_mun_2','spatial_max_p_mun_3']

s_num_clusters = []
s_mun_mean = []
s_mun_min = []
s_mun_max = []
p_silhouette = []


for m in spatial_method:
    s_n_clusters = len(mun_geo_merged_clean_name[m].unique())
    s_m_mean = mun_geo_merged_clean_name.groupby(m)['MPIO_CDPMP'].count().mean()
    s_m_min = mun_geo_merged_clean_name.groupby(m)['MPIO_CDPMP'].count().min()
    s_m_max = mun_geo_merged_clean_name.groupby(m)['MPIO_CDPMP'].count().max()
    df_silhouette[m] = esda.path_silhouette(mun_geo_merged_clean_name[non_spatial].values,mun_geo_merged_clean_name[m], w)
    mean_p_sil_score = esda.path_silhouette(mun_geo_merged_clean_name[non_spatial].values,mun_geo_merged_clean_name[m], w).mean()
    
    s_num_clusters.append(s_n_clusters)
    s_mun_mean.append(s_m_mean)
    s_mun_min.append(s_m_min)
    s_mun_max.append(s_m_max)
    p_silhouette.append(mean_p_sil_score)

# add the results to a new column of the DataFrame
df_spatial_results['Number of clusters'] = s_num_clusters
df_spatial_results['Municipalities mean'] = s_mun_mean
df_spatial_results['Municipalities minimum'] = s_mun_min
df_spatial_results['Municipalities maximum'] = s_mun_max
df_spatial_results['Mean path silhouette score'] = p_silhouette


df_spatial_results

In [ ]:
df_silhouette_clean_name = df_silhouette.rename(columns={spatial_method[0]: 'Population threshold - 100K',
                                                         spatial_method[1]: 'Population threshold - 500K',
                                                spatial_method[2]:'Population threshold - 1000K',
                                                spatial_method[3]:'Municipality threshold - 5',
                                                spatial_method[4]:'Municipality threshold - 15',
                                                spatial_method[5]: 'Municipality threshold - 25'})
df_silhouette_clean_name.head()

In [ ]:
df_silhouette_clean_name_hist = df_silhouette_clean_name.hist(figsize = (20,10),bins=75)
plt.suptitle('Histograms path silhouette score')
#ax.set_title("test")
plt.subplots_adjust(hspace=0.4, wspace=0.4)
plt.show()

## Discussion

## Conclusion

## References

Andrés, F., Arteaga, N., Arteaga, G., Llinás, D., Andrés, M., Jorge, A., Rueda-Gallardo, J., and Villamil. (2015). ‘Definición de Categorías de Ruralidad [Definition of Rurality Categories]’. Archivos de Economía.


DANE. (2021). Geoportal DANE - Descarga del Marco Geoestadistico Nacional (MGN) [DANE Geoportal - Download the National Geostatistical Framework (MGN)]. Available at: https://geoportal.dane.gov.co/servicios/descarga-y-metadatos/descarga-mgn-marco-geoestadistico-nacional/ (Accessed: 23 April 2023).


Duranton, G. (2015). ‘Roads and trade in Colombia’. Economics of Transportation. (Special Issue on Collective Contributions in the Honor of Richard Arnott), 4 (1), pp. 16–36. doi: 10.1016/j.ecotra.2014.11.003.


Gáfaro, M. and Pellegrina, H. S. (2022). ‘Trade, farmers’ heterogeneity, and agricultural productivity: Evidence from Colombia’. Journal of International Economics, 137, p. 103598. doi: 10.1016/j.jinteco.2022.103598.


Sanchez, C. C., González, D. S. and Villamil, I. O. (2015). ‘Tipologías Departamentales y Municipales: una propuesta para comprender las entidades territoriales colombianas [Departmental and Municipal Typologies: a proposal to understand Colombian territorial entities]’. Available at: https://colaboracion.dnp.gov.co/CDT/Estudios%20Econmicos/2015may28%20Tipologias.pdf (Accessed: 3 April 2023).


Sánchez, F. and Núñez’, J. (2000). ‘Geography and Economic Development: A Municipal Approach for Colombia’.





In [ ]:
print('Complete run time: ' + str((timeit.default_timer() - start_time)/60)+' minutes.')